# Simplifing agentic workflows
Agentic workflows suffer from non-deterministic model outputs and error propagation. If just a single LLM prompt fails in a long agentic workflow, the overall result may be compromised. As checking intermediate results is not always easy, it might make sense to trade flexibility of agentic workflow with determinism by exchanging parts of the agentic workflow with classical text processing approaches. This notebook demonstrates that the entire agentic workflow for writing a scientific review, shown before, can also be implemented with a single prompt.

In [1]:
from IPython.display import display, Markdown
from arxiv_utilities import prompt_scadsai_llm, get_arxiv_metadata

In [2]:
model = "meta-llama/Llama-3.3-70B-Instruct"
prompt = prompt_scadsai_llm
verbose = True

## Accumulating paper contents
Here we use a for-loop to collect paper contents in a string. As mentioned before, we are just collecting paper abstracts due to technical limitations of state-of-the-art LLMs. Token limits prevent us from collecting entire papers in one long string.

In [3]:
def read_arxiv_paper(arxiv_url:str)->str:
    """Read the abstract of an arxiv-paper and return most important contents in markdown format.

    Args:
        arxiv_url: url of the Arxiv paper
    """
    if verbose:
        print(f"read_arxiv_paper({arxiv_url})")
    arxiv_id = arxiv_url.split("/")[-1]
    metadata = get_arxiv_metadata(arxiv_id)
    title = metadata["title"]
    summary = metadata["summary"]
    authors = ", ".join(metadata["authors"])
    
    return f"""## {title}
By {authors}

{summary}
"""

In [ ]:
paper_urls = ["https://arxiv.org/abs/2211.11501",
"https://arxiv.org/abs/2308.16458",
"https://arxiv.org/abs/2411.07781",
"https://arxiv.org/abs/2408.13204",
"https://arxiv.org/abs/2406.15877"]

paper_contents = ""
for url in paper_urls:
    paper_contents += read_arxiv_paper(url) + "\n"

read_arxiv_paper(https://arxiv.org/abs/2211.11501)
read_arxiv_paper(https://arxiv.org/abs/2308.16458)


## Prompting
Here we combine the paper contents with detailed instructions for writing a manuscript reviewing those papers.

In [ ]:
result = prompt(f"""You are a great scientific writer. Your task is to write a scientific review manuscript about some papers which are summarized below.

# Content

Write about the following summarized papers:

{paper_contents}

# Formatting
Your goal is to write a manuscript that follows these criteria:
* a scientific text with a short and descriptive title,
* a scientific text with markdown sub-sections (# title, ## headlines, ...) avoiding bullet points,
* structured in sub-sections by content, e.g. introduction, recent developments, methods, results, discussion, future work, ...
* text using high-quality scientific language,
* proper citations using markdown links to original paper urls (do not make up references!),
* a clear abstract at the beginning of the text, and conclusions at the end

# Your task
Write a scientific review manuscript about the content summarized above following the mentioned formatting guidelines.
""", model=model)

In [ ]:
display(Markdown(result))

## Exercise
Use a [relexion](https://arxiv.org/abs/2303.11366) approach to give feedback to the LLM about the text it just wrote and ask it to improve the manuscript.